<a href="https://colab.research.google.com/github/ThreeMachineExpression/contrapuntalPoetry/blob/main/PhoneticSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Parts of this notebook are based on Max Woolf's aitextgen notebook, as updated by Allison Parrish.

Also makes use of Kyle Gorman's syllabification library, syllabify.

Initial implementation looked for slant rhyme / slant alliteration by grabbing all of the consonants from the boundaries between vowels and summing their features, but that leads to too-mushy results. The syllabification approach misses assonance that crosses syllable boundaries - a todo is to figure out how to bring some of that back.

Todo: correctly handle single letters (as in e.g.). It sounds them out correctly, but adds two STOPs between each for the periods.

### Setup

In [1]:
# Freeze versions of dependencies for now
!pip install tensorflow==1.15.0 keras==2.2.5 "h5py<3.0.0"
!pip3 install pytorch-lightning==0.7.6
!pip3 install transformers==2.9.1
!pip3 install fire==0.3.0

!pip install -q aitextgen==0.2.3

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

mount_gdrive()

!pip install annoy
!pip install pronouncing
!pip install pincelate
import pronouncing
import pincelate
pin = pincelate.Pincelate()

from collections import Counter

import random
import textwrap
import numpy as np

import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )

kPhoneticSimilarityVectorsRepo = "https://github.com/aparrish/phonetic-similarity-vectors"

!git clone {kPhoneticSimilarityVectorsRepo}

!python -m spacy download en_core_web_md

%cd phonetic-similarity-vectors
from featurephone import phone_feature_map as pfm
%cd ..

kSyllabifyRepo = "https://github.com/threemachineexpression/syllabify"
!git clone {kSyllabifyRepo}

%cd syllabify
import syllabify
%cd ..

     |████████████████████████████████| 412.3MB 28kB/s 
     |████████████████████████████████| 337kB 44.5MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 512kB 49.8MB/s 
     |████████████████████████████████| 3.8MB 46.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=91498d9cc11a3684b59fce5857da7fcd550f06c4475b4079fba10ef7d060ae25
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.0
    Un

Using TensorFlow backend.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Cloning into 'phonetic-similarity-vectors'...
remote: Enumerating objects: 32, done.
remote: Total 32 (delta 0), reused 0 (delta 0), pack-reused 32
Unpacking objects: 100% (32/32), done.
     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051304 sha256=2ca5ad03e7057ed1701147eff9cc324b8ccce1743514e62345db2878f5be0969
  Stored in directory: /tmp/pip-ephem-wheel-cache-tuf9uc4_/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
/content/phonetic-similarity-vectors
/content
Cloning into 'syllabify'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 62 (delta 3), reused 0 (delta 0), pack-reused 53
Unpacking objects: 100% (62/62), done.
/content/syllabify
/con

In [2]:
from_folder = "aitextgenDreamFineTuning"

for file in ["pytorch_model.bin", "config.json"]:
  if from_folder:
    copy_file_from_gdrive(file, from_folder)
  else:
    copy_file_from_gdrive(file)

In [3]:
ai = aitextgen(model="pytorch_model.bin", config="config.json", to_gpu=True)

INFO:aitextgen:Loading GPT-2 model from provided pytorch_model.bin.
INFO:aitextgen:Using the default GPT-2 Tokenizer.


# Generation

In [ ]:
!nvidia-smi

Sun Jan 24 02:49:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    40W / 300W |  16087MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
ai.generate()


You've made my night so much easier.
Oh, I'm so sorry.
I'm so sorry.
I'm so sorry.
I know you didn't mean it, but I know you didn't.
I was just so sorry I couldn't get out from under you.
And I could be kicked out of Eastman for it, but really I could be kicked out for anything, I'm already sneezing the wrong way, I'm starving, and having a really bad case of scabies, and I wake up the next morning thinking I've been impregnated, and it wouldn't have happened if I hadn't had the thought that what you were thinking.
So I'm trying really hard to think of what I'm going to say to get out of this, to make sure no one sees it, to make sure that no one sees my weaknesses, to make sure that the people who love me don't hurt me when I say it, to make sure that the damage isn't irreparable.
I'm trying really hard to pretend that I don't know anything about this kid, that I know how special this kid is and how talented this kid is, that I don't have to see his face in real


# Phonetic Similarity

In [4]:
# Strategy:
# Split text into syllable buckets
# In each bucket, include the vowel sound, stress, and all of the bordering phonemes
#  (so a consonant between syllables goes in both buckets)
# Measure the distance between two buckets as 
#   a * (manhattan distance of consonant features)
# + b * (manhattan distance of vowel features)
# + c * (stress distance)
# (with a, b, c tuned as desired to emphasize different similarities)
#
# Pauses get a bucket of their own.



class Syllable:
  # initialization takes a list of 3 lists of strings:
  # the phones for onset, nucleus, and coda
  # (as per the output of syllabify).
  #
  # vowel - vowel phone (w/o stress indicator) or STOP
  # stress - 0 for STOP, 1 for unstressed, 2 2ndary stress, 3 primary stress
  # (note this is different from the arpabet numbers)
  # ofeat - Counter containing a count of the consonant features in the onset
  # cfeat - Counter containing a count of the consonant features in the coda
  # vfeat - Counter containing a count of the vowel features
  def __init__(self, syllableList):
    self.vowel = syllableList[1][0][0:-1]

    # convert arpabet stress to a representation capable of similarity math
    stressConversion = {
        '0': 1,
        '1': 3,
        '2': 2
    }

    if self.vowel == 'STOP':
      self.stress = 0
    else:
      self.stress = stressConversion[syllableList[1][0][-1]]
      
    self.ofeat = Counter()
    self.cfeat = Counter()
    self.vfeat = Counter()

    for feature in pfm[self.vowel]:
      self.vfeat[feature] += 1
    
    for phone in syllableList[0]:
      for feature in pfm[phone]:
        self.ofeat[feature] += 1
    
    for phone in syllableList[2]:
      for feature in pfm[phone]:
        self.cfeat[feature] += 1

pfm['STOP'] = ()
stopSyllable = Syllable(([],['STOP0'],[]))

vowels = ('AO','AA','IY','UW','EH','IH','UH','AH','AE','EY','AY','OW','AW','OY',
          'ER','STOP')
consonantFeatures = ('alv','apr','asp','blb','dnt','frc','glt','lat','lbd','lbv','nas','pal','pla','stp','vcd','vel','vls')
vowelFeatures = ('bck','cnt','fnt','hgh','lmd','low','mid','rnd','rzd','smh','umd','unr','vwl')
doublestops = ['.','?','!',':','--']
singlestops = [',',';','- ']

# When deciding how close two syllables are, these constants indicate how much
# weight to give vowel features, stress, and consonant features
kDistanceWeightVowels = 3.0
kDistanceWeightStress = 3.0
kDistanceWeightOnset = 2.0
kDistanceWeightCoda = 5.0

def phones_for_word_fb(word):
  """Phones for word - either 1st entry in the CMU pronouncing dictionary
  or fallback to Pincelate sound-out if it's not in the dictionary.

  Takes the first pronounciation in the dictionary if there are multiple.

  Returns a list of phones.

  word - lowercase word, no spaces or punctuation
  """
  try:
    return pronouncing.phones_for_word(word)[0].split()
  except IndexError:
    r = pin.soundout(word)
    # Throw in an extra 'EH0' if there are no vowels in the mix already
    for v, p in [(v,p) for v in vowels for p in r]:
      if v in p:
        return r
    r.append('EH0')
    return r

def syllablesFromText(string):
  """Turns text into a list of Syllable objects.

  One stop for ,; two stops for .:?! or double dash.
  Turns a hyphen between letters (as in "use-case") into a space,
   but in "a phrase - like this one" replaces it with a STOP

  Discards all other punctuation and non-alpha characters (including numbers).

  string -- text to convert to phones"""

  lidx = ridx = 0
  
  syllables = []

  while (lidx < len(string)):
    if string[lidx].isalpha():
      while ((ridx < len(string)) and (string[ridx].isalpha())):
        ridx += 1
      for s in syllabify.syllabify(phones_for_word_fb(string[lidx:ridx].lower())):
        syllables.append(Syllable(s))
      lidx = ridx
    else:
      while ((ridx < len(string)) and (not string[ridx].isalpha())):
        ridx += 1
      doublestop = False
      if any(c in string[lidx:ridx] for c in doublestops):
        syllables.extend([stopSyllable, stopSyllable])
      else:
        if any(c in string[lidx:ridx] for c in singlestops):
          syllables.append(stopSyllable)
      lidx = ridx

  return syllables


                
def distance(a : Syllable, b : Syllable):
  """A distance metric between two Syllable objects.
  Adjusted by the constant weights in the first cell,
  applied to Manhattan distance on features.
  """ 
  dist = abs(a.stress - b.stress) * kDistanceWeightStress

  for f in consonantFeatures:
    dist += abs(a.ofeat[f] - b.ofeat[f]) * kDistanceWeightOnset
    dist += abs(a.cfeat[f] - b.cfeat[f]) * kDistanceWeightCoda
  
  for f in vowelFeatures:
    dist += abs(a.vfeat[f] - b.vfeat[f]) * kDistanceWeightVowels

  return dist

def isVowel(phone):
  return phone[0:-1] in vowels

def canonFit(oldSyllables, newText, offset):
  """Given a syllable list and a batch of new text (as a string),
  returns the average syllable distance of newly added syllables
   to syllables *offset* syllables behind.
  
  Lower is rhymier.
  
  If newText doesn't turn up any new syllables, return a stupidly big number.

  Warning: GPT-2 continuations can add partial words or just whitespace or
  punctuation. This method makes no effort to re-syllabize the end of
  oldSyllables or use that information to pronounce the beginning of newText.
  Prepare data along word boundaries before passing to this method.
  """

  newSyllables = syllablesFromText(newText)

  if len(newSyllables) == 0:
    return float("inf")
  
  existingLength = len(oldSyllables)
  totalDist = i = averageCounter = 0

  for s in newSyllables:
    idx = existingLength - offset + i
    # don't look back past the beginning of the text
    if idx >= 0:    
      if idx < existingLength:
        t = oldSyllables[idx]
      else:
        t = newSyllables[idx - existingLength]
      totalDist += distance(s, t)
      averageCounter += 1
    i += 1
  
  if averageCounter > 0:
    return totalDist/averageCounter
  else:
    # haven't hit the canon start point, or only added STOPs that match up with STOPs;
    # accept this continuation and keep going
    # (theoretical risk of getting stuck in STOPland - we'll see if that's a real problem)
    return 0

def isascii(string):
  # Apparently this is a fast way of checking because the conversion
  # is implemented in C
  try:
    string.encode('ascii')
  except UnicodeEncodeError:
    return False
  else:
    return True

In [29]:
class Continuations:
  def __init__(self, alts, prompt, max_length, temperature, top_p):
    self.prompt = prompt
    self.max_length = max_length
    self.temperature = temperature
    self.top_p = top_p

    self.leftToGenerate = alts
    self.counter = 0
    self.continuationsToTry = 256
    self.cacheIter = iter([])

  def __iter__(self):
    return self

  def __next__(self):
    try:
      return self.cacheIter.__next__()
    except StopIteration:
      if self.leftToGenerate == 0:
        raise(StopIteration)
        
      while self.continuationsToTry > 1:
        try:
          n = min(self.leftToGenerate, self.continuationsToTry)
          self.cacheIter = iter(ai.generate(n=n,
                                       prompt=self.prompt,
                                       max_length=self.max_length,
                                       temperature=self.temperature,
                                       top_p=self.top_p,
                                       return_as_list=True))
          self.leftToGenerate = max(0, self.leftToGenerate - n)
          return self.cacheIter.__next__()
        except RuntimeError as e:
          print("Runtime Error, reducing batch size: " + str(e))
          self.continuationsToTry = self.continuationsToTry//2
      
      self.cacheIter = iter(ai.generate(n=self.continuationsToTry,
                                       prompt=self.prompt,
                                       max_length=self.max_length,
                                       temperature=self.temperature,
                                       top_p=self.top_p,
                                       return_as_list=True))
      return self.cacheIter.__next__()


def generateCanon(alts = 100, tokensPerIncrement = 5, length = 100,
                  prompt = "If only I could be a little more",
                  temperature = 1.2, top_p = 0.9, offset = 10):
  """
  alts - number of samples of text to generate to test for rhymes
  tokensPerIncrement - size of each incremental sample
  length - total tokens (including the prompt)
  prompt - start of canon
  temperature, top_p - passed along to GPT-2
  offset - how many syllables the 2nd part of the canon is delayed
  """
  print(prompt)
  canonSoFar = prompt
  seed = 0
  startingTokens = len(ai.tokenizer.tokenize(text=prompt))



  for i in range(startingTokens, length, tokensPerIncrement):
    # Tricky thing - GPT-2's tokens are sometimes non-words and partial words.
    # "don" might end a previous pass and be expanded to "don't" in a continuation.
    # Or a continuation could add only line breaks and punctuation.    
    # So we split canonSoFar up to the last non-alpha character.
    # We'll attach the rest of canonSoFar to the newly generated text.
    lastNonAlphaInCanonSoFar = len(canonSoFar) - 1
    while lastNonAlphaInCanonSoFar > 0:
      if not canonSoFar[lastNonAlphaInCanonSoFar].isalpha():
        break
      lastNonAlphaInCanonSoFar = lastNonAlphaInCanonSoFar - 1
    
    solidCanon = canonSoFar[0:lastNonAlphaInCanonSoFar]
    tentativeCanon = canonSoFar[lastNonAlphaInCanonSoFar:]
    
    solidCanonSyllables = syllablesFromText(solidCanon)  
    
    bestTrial = canonSoFar
    bestDist = float("inf")

    continuations = Continuations(alts, canonSoFar, i, temperature,
                                  top_p)
    for trial in continuations:
      newText = tentativeCanon + trial[len(canonSoFar):]
      if (isascii(newText)  # only test generated text that our pronounciation lookups can handle
          and not any(c.isdigit() for c in newText)):  # discard text with numbers 
        trialFit = canonFit(solidCanonSyllables, newText, offset)
        if trialFit < bestDist and trialFit > 0: # discourage exact loops
          bestDist = trialFit
          bestTrial = trial
    
    canonSoFar = bestTrial
    print()
    print(canonSoFar)

  print()
  print("FINAL CANON:")
  print(canonSoFar)

In [ ]:
generateCanon(alts = 250, tokensPerIncrement=2, length=200, prompt="A transition matrix", offset = 10, temperature = .5, top_p = 0.9)


A transition matrix
Generating 250 continuations

A transition matrix
Generating 250 continuations
Runtime Error, reducing batch size: CUDA out of memory. Tried to allocate 192.00 MiB (GPU 0; 15.90 GiB total capacity; 14.70 GiB already allocated; 25.88 MiB free; 14.91 GiB reserved in total by PyTorch)
Generating 128 continuations
Generating 122 continuations

A transition matrix
Generating 250 continuations
Runtime Error, reducing batch size: CUDA out of memory. Tried to allocate 192.00 MiB (GPU 0; 15.90 GiB total capacity; 14.70 GiB already allocated; 25.88 MiB free; 14.91 GiB reserved in total by PyTorch)
Generating 128 continuations
Generating 122 continuations

A transition matrix is in use.
Generating 250 continuations
Runtime Error, reducing batch size: CUDA out of memory. Tried to allocate 336.00 MiB (GPU 0; 15.90 GiB total capacity; 14.57 GiB already allocated; 25.88 MiB free; 14.91 GiB reserved in total by PyTorch)
Generating 128 continuations
Runtime Error, reducing batch siz

with new syllabification, temperature 5, top_p .2, tokensPerIncrement = 5, offset 5, params 4/5/2/3 (vowels/stress/onset/coda)

```
If only I could be in a meeting coordinating this, explaining things.
Getting these people to believe I can't deliver on time has never made any difference, even in the event that they're irate and cry when I'm explaining why we didn't, I haven't, I haven't, it never was; it never will, it never will; if I were building this case, building this case, building a case about me and [1000Plateaus] I probably would have had a clearer shot at what people thought I was doing than I was.
I am as bad as I think myself as I can be when it happens to me - I can't be in an arty tent in an alleyway in an arty prison, hardly eating, hardly paying my tab, staring at strangers ferried here from the slobbery of the north, a dozen screaming asylum-climbers and clinging desperately to their property - I have plenty of time, money
```



```
If  only I  could be in that  boat.       That boat.
            If    only  I     could be in that boat.

I    don't know why.       Cousinwog, cousin of  Brassy and
That boat.         I don't know why.  Cousinwog,     cousin 

Red from    Cousinwog  and   Mighty from   Cousinwog  
of  Brassy     and Red from  Cousinwog    and Mighty from

City  and  Plateaus  aren't  together        anymore.   They're
Cousinwog  City      and     Plateaus aren't together anymore. 

having a breakout and Plateaus says "What's going    on    here?"
      They're having   a breakout    and    Plateaus says "What's

going on here?"
```

(offset of 4, temperature .7)




```
If only I  could be my good   editor     in one piece 
        If only     I  could  be my good editor

-- which, at least,   it   was, when  absolutely forced and wri-
editor   --  which at least,    it   was, when absolutely

ting   ridiculously    long chunks of crap chunks.
forced and writing ridiculously       long chunks of crap

Without the   ethics.          "It sucks."    Sometimes,
chunks.       Without  the  ethics.       "It sucks."

sometimes, it's just "it's difficult       to tell a
Sometimes, sometimes,      it's just "it's difficult

story"   as art,   or an illusion.    There's another,     ever
to tell  a  story" as art, or an illusion.       There's another,

another trance.  Sometimes it's "I don't  think it's
   ever another  trance.        Sometimes it's  "I

important"         as   entertainment.
don't think it's   important" as  entertainment.
```
(offset of 3, temperature 2.0, vowels 4, stress 3, consonants 5)


```
If only I could be allowed to  be loved.     Been  avoiding
          If    only  I  could be allowed to be    loved.

eye  doctor since  ninety-two.
Been avoiding      eye doctor  since ninety-two.
```

(offset of 4, temperature 2, vowels, stress, cons = 2,1,2; 3 tokens per increment)

```
If only I could be that good.
People say it - "It's true it isn't" - "It is", "It is true it is true" - "Dear @lukejchengnott, in what way?"
```

(same settings, but offset of 3)

```
If only
I could be
as good as
this.
I know I'm
doing ev
ery thing right.
          I
know I'm do
ing every
thing I can.
          I
know I'm do
ing every
thing I can.
          I
know I'm cap
able.
     I be
lieve I am
loved.
I believe
I am ent
ertaining
the impos
sible.
  But all
of a   sud
den it seems
like it's a
bout to col
lapse on it
self and I
have to find
a place to
stand,  so
I don't lis
ten.
Finally
I find a
place to sit,
   thinking
it's okay
to be there.
      There
is a rea
sonable
place to go,
    but I'm
worried it
will be rai
ded by the
big demon
ic fuckups,
   and killed.
          I
know that I
am allowed
to feel my
anger and
anxie
ty under
control,
that this is
an extreme
ly danger
ous place.
    I sit
on it,
thinking it's
really o
kay to be
there.
But then I
get really
worried and
angry   and
a     pile  of
Trash piles up.
```

(alts -> 50 from 250, 5 tokens per increment, temperature 1)

(stress weight 1->200, offset -> 3)

30.0

In [ ]:
kDistanceWeightVowels = 3.0
kDistanceWeightStress = 3.0
kDistanceWeightOnset = 2.0
kDistanceWeightCoda = 5.0


In [ ]:
syllabify.syllabify(phones_for_word_fb("transmute"))

[(['T', 'R'], ['AE0'], ['N', 'S']), (['M'], ['Y', 'UW1'], ['T'])]

In [ ]:
kDistanceWeightVowels = 3.0
kDistanceWeightStress = 3.0
kDistanceWeightOnset = 2.0
kDistanceWeightCoda = 5.0


In [ ]:
kDistanceWeightCoda

3.0